In [12]:
import datetime
import pandas as pd

In [96]:
df = pd.read_csv("local/games.csv", parse_dates=["GAME_DATE_EST"], dtype={
    "HOME_TEAM_ID": str,
    "VISITOR_TEAM_ID": str,
    "GAME_ID": str,
    "SEASON": str,
    "HOME_TEAM_WINS": str
})

# Drop redundant columns and entries with null columns
df = df.drop(["GAME_STATUS_TEXT", "TEAM_ID_home", "TEAM_ID_away"], axis=1)
df = df.dropna()

# Sort by ascending date
df = df.sort_values("GAME_DATE_EST")

# Set game date as index
df = df.set_index("GAME_DATE_EST", drop=False)

df.head()

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
GAME_DATE_EST,,,,,,,,,,,,,,,,,,
2003-10-05,2003-10-05,10300001,1610612762,1610612742,2003,90.0,0.457,0.735,0.143,23.0,41.0,85.0,0.447,0.500,0.250,20.0,38.0,1
2003-10-06,2003-10-06,10300002,1610612763,1610612749,2003,105.0,0.494,0.618,0.267,25.0,48.0,94.0,0.427,0.700,0.154,20.0,43.0,1
2003-10-07,2003-10-07,10300010,1610612764,1610612752,2003,104.0,0.506,0.677,0.455,26.0,45.0,86.0,0.380,0.852,0.188,19.0,37.0,1
2003-10-07,2003-10-07,10300009,1610612758,1610612746,2003,101.0,0.467,0.871,0.444,19.0,39.0,82.0,0.368,0.609,0.364,13.0,50.0,1
2003-10-07,2003-10-07,10300005,1610612757,1610612745,2003,104.0,0.527,0.657,0.429,22.0,33.0,80.0,0.470,0.667,0.333,10.0,37.0,1


In [107]:
# # Filter only columns that may be used during prediction
# df_training = df[["GAME_DATE_EST", "HOME_TEAM_ID", "VISITOR_TEAM_ID", "SEASON", "HOME_TEAM_WINS"]]

# df_home = df[["GAME_DATE_EST", "HOME_TEAM_ID", "PTS_home", "FG_PCT_home", "FT_PCT_home", "FG3_PCT_home", "AST_home", "REB_home"]]
# df_away = df[["GAME_DATE_EST", "VISITOR_TEAM_ID", "PTS_away", "FG_PCT_away", "FT_PCT_away", "FG3_PCT_away", "AST_away", "REB_away"]]

# # def add_historical_data(row):
# #     df_home_prev = df_home[df_home["HOME_TEAM_ID"] == row["HOME_TEAM_ID"]].loc[:row["GAME_DATE_EST"]].last('1D')
# #     df_away_prev = df_away[df_away["VISITOR_TEAM_ID"] == row["VISITOR_TEAM_ID"]].loc[:row["GAME_DATE_EST"]].last('1D')
# #     return [*df_home_prev, *df_away_prev]

# # # df_training = df_training.apply(lambda r: add_historical_data(r), axis=1)
# # df_training[[
# #     "PTS_home", "FG_PCT_home", "FT_PCT_home", "FG3_PCT_home", "AST_home", "REB_home",
# #     "PTS_away", "FG_PCT_away", "FT_PCT_away", "FG3_PCT_away", "AST_away", "REB_away"
# # ]] = df_training.apply(add_historical_data, axis=1, result_type="expand")

# df_by_date = df.groupby("GAME_DATE_EST").shift(-1)
df_by_team = df.reset_index(drop=True).groupby(["HOME_TEAM_ID", "VISITOR_TEAM_ID"]).size()
df_by_team

HOME_TEAM_ID  VISITOR_TEAM_ID
1610612737    1610612738         44
              1610612739         41
              1610612740         26
              1610612741         40
              1610612742         21
                                 ..
1610612766    1610612761         32
              1610612762         19
              1610612763         20
              1610612764         41
              1610612765         37
Length: 870, dtype: int64